In [1]:
import pymongo
client = pymongo.MongoClient("localhost", 27017) # Connect to the db server
db = client.bilibili
db1 = db['videos0-5m+']
db1.create_index(("VideoID"),unique=True)

u'VideoID_1'

In [2]:
urls = []
for i in range(0,1000000):
    url = 'http://bilibili.com/video/av' + str(i)
    urls.append(url)

In [3]:
# -*-coding:utf8-*-

from lxml import etree
from multiprocessing.dummy import Pool as ThreadPool
import requests
import time
import sys
import re
import json


reload(sys)

sys.setdefaultencoding('utf-8')

# id av cid title tminfo time click danmu coins favourites duration honor_click honor_coins honor_favourites
# mid name article fans tags[3] common


head = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36'
}

def spider(url):
    html = requests.get(url, headers=head)
    selector = etree.HTML(html.text)
    content = selector.xpath("//html")
    for each in content:
        title = each.xpath('//div[@class="v-title"]/h1/@title')
        if title:
            av = url.replace("http://bilibili.com/video/av", "")
            title = title[0]
            tminfo1_log = each.xpath('//div[@class="tminfo"]/a/text()')
            tminfo2_log = each.xpath('//div[@class="tminfo"]/span[1]/a/text()')
            tminfo3_log = each.xpath('//div[@class="tminfo"]/span[2]/a/text()')
            if tminfo1_log:
                tminfo1 = tminfo1_log[0]
            else:
                tminfo1 = ""
            if tminfo2_log:
                tminfo2 = tminfo2_log[0]
            else:
                tminfo2 = ""
            if tminfo3_log:
                tminfo3 = tminfo3_log[0]
            else:
                tminfo3 = ""
            tminfo = tminfo1 + '-' + tminfo2 + '-' + tminfo3
            time_log = each.xpath('//div[@class="tminfo"]/time/i/text()')
            mid_log = each.xpath('//div[@class="usname"]/a/@mid')
            username_log = each.xpath('//div[@class="usname"]/a/@title')
            if time_log:
                time = time_log[0]
            else:
                time = ""
            if mid_log:
                mid = mid_log[0]
            else:
                mid = ""
            if username_log:
                username = username_log[0]
            else:
                username = ""

            cid_html_1 = each.xpath('//div[@class="scontent"]/iframe/@src')
            cid_html_2 = each.xpath('//div[@class="scontent"]/script/text()')
            if cid_html_1 or cid_html_2:
                if cid_html_1:
                    cid_html = cid_html_1[0]
                else:
                    cid_html = cid_html_2[0]

                cids = re.findall(r'cid=.+&aid', cid_html)
                cid = cids[0].replace("cid=", "").replace("&aid", "")
                info_url = "http://interface.bilibili.com/player?id=cid:" + str(cid) + "&aid=" + av
                video_info = requests.get(info_url)
                video_selector = etree.HTML(video_info.text)
                for video_each in video_selector:
                    click_log = video_each.xpath('//click/text()')
                    duration_log = video_each.xpath('//duration/text()')
                    if click_log:
                        click = click_log[0]
                    else:
                        click = -1
                    if duration_log:
                        duration = duration_log[0]
                    else:
                        duration = ""
                try:
                    video = {'VideoID':str(av), 'cid':cid, 'title':title, 'catalog':tminfo, 'time':time, 'click':click,
                            'duration':duration, 'posterid':mid, 'postername':username}
                    db1.insert_one(video)
                    
                except :
                    print "Error "
                print "Succeed: av" + str(av)
            else:
                print "Error_noCid:" + url
        else:
            print "Error_404: " + url




In [ ]:
for i in range(3,10000):
    urls = []
    a = (0+100*(i-1))
    b = (0+100*(i))
    for k in range(a,b):
        url = 'http://bilibili.com/video/av' + str(k)
        urls.append(url)
    print a,'to',b,'has been crawled'
    pool = ThreadPool(8)
    try:
        results = pool.map(spider, urls)
    except Exception, e:
        print e
        time.sleep(300)
        results = pool.map(spider, urls)
    pool.close()
    pool.join()

In [4]:
pool = ThreadPool(8)
try:
    results = pool.map(spider, urls)
except Exception, e:
    print e
    time.sleep(300)
    results = pool.map(spider, urls)
pool.close()
pool.join()

IndexError: list index out of range